## Preambule

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Read data

In [2]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
xr_nld_raw = xr.open_dataset("K:/ECEMF/T5.2/Allocations/xr_alloc_NLD.nc")
rules = ['GF', 'PCC', 'PC', 'AP', 'ECPC', 'GDR']

In [3]:
df = pd.read_excel("X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [4]:
reds_2040 = np.zeros(shape=(len(group_eu), 6))
for cty_i, cty in enumerate(group_eu):
    emis_1990 = xr_dataread.sel(Time=1990).GHG_hist.sel(Region=cty)
    xr_cty = xr.open_dataset("K:/ECEMF/T5.2/Allocations/xr_alloc_"+cty+".nc").sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.5, Risk=0.5, Scenario='SSP2', Convergence_year=2050, Timing='Immediate')
    emis_2040 = xr_cty.sel(Time=2040)[rules]
    reds_2040[cty_i] = -((emis_2040 - emis_1990)/emis_1990).to_array()

In [5]:
reds_2040_n = (reds_2040-reds_2040[group_eu=='NLD']) / reds_2040[group_eu=='NLD']
avreds = np.nanmean(reds_2040_n, axis=1)

In [6]:
reds_2040_eu = np.nanmean(reds_2040, axis=0)
reds_2040_p = (reds_2040) / reds_2040_eu

In [7]:
rulecolors = np.array([(90, 51, 36), (141, 145, 31), (0, 156, 223), (157, 0, 100), (230, 173, 31), (61, 29, 92), ])
rulenames = ['Grandfathering', 'Per capita convergence', 'Immediate per capita', 'Ability to pay', "Equal cumulative\nper capita", 'Greenhouse Development\nRights']
rulecolors_hex = ["#{0:02x}{1:02x}{2:02x}".format(i[0], i[1], i[2]) for i in rulecolors]

## Plot

In [14]:
fig = make_subplots(rows=3, cols=5, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type':'polar'}, {'type': 'choropleth', 'colspan': 3, 'rowspan': 3}, {}, {}, {'type':'polar'}], [{'type':'polar'}, {}, {}, {}, {'type':'polar'}], [{'type':'polar'}, {}, {}, {}, {'type':'polar'}]],
                                    horizontal_spacing = 0.,
                                    vertical_spacing=0.1)
                    # insets=[dict(cell=(1,1), l=0.55, b= 0.43),
                    #         dict(cell=(1,1), l=0., h=0., b=0.,  type='polar')])

fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = avreds,
    locationmode = 'ISO-3',
    colorscale = 'RdBu_r',
    zmax = 0.25,
    zmin = -0.25,
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='black', 
    marker_line_width=1,
    colorbar=dict(len=1, x=1, tickvals = np.arange(-25, 25+0.1, 5), title={"text":"GHG budget<br>2021-2100<br>(Gt CO<sub>2/sub>e)"}, tickfont={'size': 25}),
    showscale=False
), 1, 2)
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
)
vals = (avreds*100).round(1)
vals_new = []
for v_i, v in enumerate(vals):
    if v > 0:
        vn = '+'+str(v)+'%'
    else:
        vn = str(v)+'%'
    vals_new.append(vn)

fig.add_scattergeo(
            locations = group_eu,
            text = np.array(vals_new),
            featureidkey="properties.NAME_3",
            mode = 'text',
            hoverinfo='skip',
            showlegend=False,
            textfont={'size': np.sqrt(np.abs(vals))+14, 'color': 'black'},
            row=1,
            col=2)

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[34,73], lonaxis_range=[-15, 35],
                showcoastlines=False,)
fig.update_geos(visible=True, showcountries=False)

fig.update_layout(height=1200, width=2300)
#fig.update_layout(margin={"t":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.99, x=0.25, text="<b>Fair reductions targets in 2040, compared to the Netherlands</b>")

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=20),
                   y=0.95, x=0.25, text='        Positive/red indicates more reductions required than in the Netherlands')

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=20),
                   y=0.93, x=0.25, text='        Average relative difference over all effort-sharing methods')
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=20),
                   y=0.91, x=0.25, text='        Polar plots indicate reductions compared to European average (inner ring)')

# Polar plots
for cty_i, cty in enumerate(['NLD', 'FRA', 'ESP', 'SWE', 'DEU', 'ITA']):
    row = [1, 2, 3, 1, 2, 3][cty_i]
    col = [1, 1, 1, 5, 5, 5][cty_i]
    cty_full = ['Netherlands', 'France', 'Spain', 'Sweden', 'Germany', 'Italy'][cty_i]
    fig.add_annotation(xref='paper', yref='paper',ax=0, align='center', ay=-0, showarrow=False, font=dict(color='black', size=18),
                       y=[1.05, 0.67, 0.29, 1.05, 0.67, 0.29][cty_i],
                       x=[0.07, 0.07, 0.07, 0.92, 0.92, 0.92][cty_i],
                       text="<b>"+cty_full+"</b>")
    for rule_i, rule in enumerate(rules):
        val = reds_2040_p[group_eu==cty][0][rule_i]
        valo = val-0.5
        if valo>1: valo = 1
        if valo<0: valo = 0
        fig.add_trace(go.Barpolar(
            r=[val],
            theta=[[i*60 for i in range(6)][rule_i]],
            width=[60],
            marker_color=rulecolors_hex[rule_i],
            marker_line_color="black",
            marker_line_width=0,
            showlegend=False,
            opacity=0.8
        ), row, col)
    fig.add_trace(go.Scatterpolar(
        r=[1]*len(np.linspace(0, 360, 1000)),
        theta=np.linspace(0, 360, 1000),
        mode='lines',
        showlegend=False,
        line_color='black',
        line_width=3
    ), row, col)

fig.update_layout(
    #template=None,
    polar = dict(
        radialaxis = dict(range=[0.8, 1.2], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks=''),
        radialaxis_gridcolor="darkgrey",
    ),
    polar3 = dict(
        radialaxis = dict(range=[0.8, 1.2], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks=''),
        radialaxis_gridcolor="darkgrey",
    ),
    polar4 = dict(
        radialaxis = dict(range=[0.8, 1.2], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks=''),
        radialaxis_gridcolor="darkgrey",
    ),
    polar5 = dict(
        radialaxis = dict(range=[0.8, 1.2], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks=''),
        radialaxis_gridcolor="darkgrey",
    ),
    polar6 = dict(
        radialaxis = dict(range=[0.8, 1.2], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks=''),
        radialaxis_gridcolor="darkgrey",
    ),
    polar2 = dict(
        radialaxis = dict(range=[0.8, 1.2], showticklabels=False, ticks=''),
        angularaxis = dict(showticklabels=False, ticks=''),
        radialaxis_gridcolor="darkgrey",
    )
)

def linshape(col, x0, y0, x1, y1, type='solid'):
    fig.add_shape(type="line", xref='paper', yref='paper',
                    x0=x0,
                    y0=y0,
                    x1=x1,
                    y1=y1,
                    line=dict(color=col, width=1, dash=type))
linshape('black', 0.1, 0.87, 0.45, 0.49)
linshape('black', 0.1, 0.5, 0.4, 0.39)
linshape('black', 0.1, 0.13, 0.3, 0.22)
linshape('black', 0.9, 0.87, 0.6, 0.8)
linshape('black', 0.9, 0.5, 0.52, 0.47)
linshape('black', 0.9, 0.13, 0.54, 0.2)

fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
fig_europe = fig
fig.show()

## Save

In [9]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/PBL_report/Fig_europe.png", scale=3)